# CryptoDataPy Walkthrough

In this notebook, we will walk through how to use `cryptodatapy` to pull data from various sources.

- **Data vendors**:
<br>
- _Data credentials_: requesting and storing your api keys in DataCredentials.
- _Metadata_: what data is available and how to find it.
- _Data requests_: parameters specifying the data you want.
- _Fetching data_: send GET request to a data vendor's API with specified parameters.
<br>
<br>

# Data Vendors

Pulling data from vendors generally requies either becoming familiar with their web APIs (by reviewing their documentation) or installing an API wrapper. 

With `cryptodatapy` you can interacct with and access data from any data vendor by using the same intuitive interface. Below, we'll use CryptoCompare as an example of how to do so. 

## Data Credentials

Most data vendors will require an API key to access their data. So before we can do so, we must first request and store our API key. 

We recommend storing API keys along with all other data credentials in DataCredentials, a dataclass created to store all relevant data/attributes for `cryptodatapy`. 

In [1]:
# import DataCredentials to set CryptoCompare API key
from cryptodatapy.util.datacredentials import DataCredentials, set_credential
# uncomment and rune line below, this only needs to be set once
# set_credential('cryptocompare_api_key', {'insert your api key here'})

Next, we can import the CryptoCompare class (or any other data vendor) and initialize an object. 
<br>
If your API key is not set, it will be returned as None from DataCredentials. You should therefore pass it as a parameter in the CryptoCompare object, as indicated below.

In [2]:
# import CryptoCompare
from cryptodatapy.data_vendors.cryptocompare import CryptoCompare
# initialize CryptoCompare
cc = CryptoCompare() # if api key set using set_credential in datacredentials.py
# else insert your cryptocompare api key, and uncomment and run the line below 
# cc = CryptoCompare(api_key={your cryptocompare api key})

Now, we can explore some of the available data...

## Metadata

`cryptodatapy`'s interface makes it easy to explore CryptoCompare's available data.

### Source Type

To get CryptoCompare's data source type

In [3]:
cc.source_type

'data_vendor'

### Categories

Categories are either assets classes or types of time series data (e.g. macro or alt data).

To get CryptoCompare's available categories

In [4]:
cc.categories

['crypto']

### Assets

To get a list of available assets

In [5]:
cc.assets

['42',
 '300',
 '365',
 '404',
 '433',
 '611',
 '808',
 '888',
 '1337',
 '2015',
 'XBS',
 'XPY',
 'PRC',
 'YBC',
 'DANK',
 'GIVE',
 'KOBO',
 'DT',
 'SUP',
 'XPD',
 'GEO',
 'NXTI',
 'WOLF',
 'XDP',
 'AEGIS',
 'ALIEN',
 'APEX',
 'ARG',
 'ARI',
 'AXR',
 'BLU',
 'BOST',
 'BQC',
 'XMY',
 'ZET',
 'SXC',
 'QTL',
 'ENRG',
 'RIC',
 'LIMX',
 'BTB',
 'CAIX',
 'BTMK',
 'BUK',
 'CACH',
 'CANN',
 'CAP',
 'CBX',
 'CCN',
 'CIN',
 'CINNI',
 'CXC',
 'CLAM',
 'CLR',
 'CNL',
 'COOL',
 'CRACK',
 'DEM',
 'DIAM',
 'DRKC',
 'DSB',
 'EAC',
 'EFL',
 'ELC',
 'EMD',
 'EXCL',
 'EZC',
 'FLAP',
 'FC2',
 'FFC',
 'FIBRE',
 'FRC',
 'FLT',
 'FRK',
 'FRAC',
 'GDC',
 'GLYPH',
 'GML',
 'GUE',
 'HAL',
 'HBN',
 'HUC',
 'HVC',
 'HYP',
 'ICB',
 'IFC',
 'IXC',
 'JBS',
 'JKC',
 'JUDGE',
 'KDC',
 'KEYC',
 'KGC',
 'LK7',
 'LKY',
 'LSD',
 'LTCD',
 'LTCX',
 'LXC',
 'LYC',
 'MAX',
 'MINRL',
 'MINC',
 'MRY',
 'MZC',
 'NAN',
 'NAUT',
 'NMB',
 'NRB',
 'NOBL',
 'NRS',
 'NYAN',
 'OPSC',
 'PHS',
 'POINTS',
 'PSEUD',
 'PXC',
 'PYC',
 'RIPO'

For more granular asset info 

In [6]:
cc.get_assets_info()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,AlgorithmType,Difficulty,BuiltOn,SmartContractAddress,DecimalPoints
ticker,,,,,,,,,,,,,,,,,,,,,
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,42,0,0,0,blockchain,scrypt,0.631212,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,NaN,NaN,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VINU,946704,/coins/vinu/overview,/media/39501395/vinu.png,1645099997,VINU,VINU,Vita Inu,Vita Inu (VINU),Vita Inu (VINU) is the governance token of the...,N/A,...,-1,0,0,0,token,NaN,NaN,"VITE,BNB,ETH",vite_1b91ee2619368ee27857c8cc544bc72e92436ba8a...,NaN
ASKO,949571,/coins/asko/overview,/media/40484591/asko.png,1656342448,ASKO,ASKO,Asko,Asko (ASKO),rASKO is an automated cross-chain money market...,N/A,...,-1,0,0,0,token,NaN,NaN,ETH,0xeEEE2a622330E6d2036691e983DEe87330588603,18
CALO,949573,/coins/calo/overview,/media/40484592/calo.png,1656343996,CALO,CALO,Calo,Calo (CALO),Calo Metaverse is a health-lifestyle platform ...,N/A,...,-1,0,0,0,token,NaN,NaN,BNB,0xb6b91269413b6B99242B1c0Bc611031529999999,18


### Indexes

To get a list of available indexes

In [7]:
cc.indexes

['MVDA', 'MVDALC', 'MVDAMC', 'MVDASC', 'BTCA', 'BVIN', 'BTCB', 'ETHB']

For more granular index info

In [8]:
cc.get_indexes_info()

,name,description,quote_currency,index_market_name,index_market_underlying_name,day_close_in_seconds
ticker,,,,,,
MVDA,MVIS CryptoCompare Digital Assets 100,,USD,CCMVDA,CCMVDA,61200
MVDALC,MVIS CryptoCompare Digital Assets 100 Large-Cap,,USD,CCMVDA,CCMVDA,61200
MVDAMC,MVIS CryptoCompare Digital Assets 100 Mid-Cap,,USD,CCMVDA,CCMVDA,61200
MVDASC,MVIS CryptoCompare Digital Assets 100 Small-Cap,,USD,CCMVDA,CCMVDA,61200
BTCA,FICAS Actively Managed ETP,,CHF,CCNAV,CCMVDA,50400
BVIN,CryptoCompare Bitcoin Volatility Index,,USD,CCMVDA_VIX,CCMVDA_VIX,36000
BTCB,FICAS Bitcoin Fund,,CHF,CCNAV,CCMVDA,61200
ETHB,FICAS Ethereum Fund,,CHF,CCNAV,CCMVDA,61200


### Markets
To get a list of availalble markets (asset:quote_ccy)

In [9]:
cc.markets

['365BTC',
 '365RUR',
 '365ETH',
 '365DOGE',
 '365USD',
 '365WAVES',
 '404XRP',
 '404USD',
 '404WAVES',
 '404RUR',
 '404ETH',
 '404DOGE',
 '888BTC',
 '2015BTC',
 'ARBTUSDT',
 'IGGBTC',
 'IGGTRX',
 'TTUUSDT',
 'TTUETH',
 'TTUBTC',
 'BTZCUSDT',
 'BTZCBTC',
 'LOGBTC',
 'ZECCKUSD',
 'ZECUSDC',
 'ZECETH',
 'ZECTUSD',
 'ZECBNB',
 'ZECBUSD',
 'ZECUSDP',
 'ZECUSDT',
 'ZECBTC',
 'ZECUSD',
 'ZECBRL',
 'ZECGBP',
 'ZECAUD',
 'ZECCPEX',
 'ZECEOS',
 'ZECEURS',
 'ZECBCH',
 'ZECHUSD',
 'ZECKRW',
 'ZECCAD',
 'ZECJPY',
 'ZECEUR',
 'ZECKCS',
 'ZECUAH',
 'ZECXVG',
 'ZECOKB',
 'ZECXRP',
 'LTCCKUSD',
 'LTCUSDT',
 'LTCBTC',
 'LTCJPY',
 'LTCAUD',
 'LTCTRY',
 'LTCTHB',
 'LTCETH',
 'LTCBNB',
 'LTCUSDC',
 'LTCEUR',
 'LTCTUSD',
 'LTCBUSD',
 'LTCNGN',
 'LTCUAH',
 'LTCGBP',
 'LTCBRL',
 'LTCILS',
 'LTCPLN',
 'LTCUSD',
 'LTCKRW',
 'LTCMXN',
 'LTCPEN',
 'LTCCLP',
 'LTCARS',
 'LTCCOP',
 'LTCBCH',
 'LTCCPEX',
 'LTCCZK',
 'LTCEURS',
 'LTCCNYT',
 'LTCRUB',
 'LTCDASH',
 'LTCDAI',
 'LTCSAI',
 'LTCEOS',
 'LTCHT',
 'LTCHUSD',

For more granular markets info

In [10]:
cc.get_markets_info()

{'365': {'BTC': {'exchanges': {'Yobit': {'isActive': True,
     'isTopTier': False}},
   'histo_minute_start_ts': 1495756800,
   'histo_minute_start': '2017-05-26',
   'histo_minute_end_ts': 1656115200,
   'histo_minute_end': '2022-06-25'},
  'RUR': {'exchanges': {'Yobit': {'isActive': True, 'isTopTier': False}},
   'histo_minute_start_ts': 1509408000,
   'histo_minute_start': '2017-10-31',
   'histo_minute_end_ts': 1656115200,
   'histo_minute_end': '2022-06-25'},
  'ETH': {'exchanges': {'Yobit': {'isActive': True, 'isTopTier': False}},
   'histo_minute_start_ts': 1510531200,
   'histo_minute_start': '2017-11-13',
   'histo_minute_end_ts': 1656115200,
   'histo_minute_end': '2022-06-25'},
  'DOGE': {'exchanges': {'Yobit': {'isActive': True, 'isTopTier': False}},
   'histo_minute_start_ts': 1509321600,
   'histo_minute_start': '2017-10-30',
   'histo_minute_end_ts': 1656115200,
   'histo_minute_end': '2022-06-25'},
  'USD': {'exchanges': {'Yobit': {'isActive': True, 'isTopTier': False}

### Market Types
 
To get a list of available market types

In [11]:
cc.market_types

['spot']

### Fields
To get a list of available fields

In [12]:
cc.fields

['open',
 'high',
 'low',
 'close',
 'volume',
 'zero_balance_addresses_all_time',
 'unique_addresses_all_time',
 'new_addresses',
 'active_addresses',
 'average_transaction_value',
 'block_height',
 'hashrate',
 'difficulty',
 'block_time',
 'block_size',
 'current_supply',
 'transaction_count',
 'transaction_count_all_time',
 'large_transaction_count',
 'comments',
 'posts',
 'followers',
 'points',
 'overview_page_views',
 'analysis_page_views',
 'markets_page_views',
 'charts_page_views',
 'trades_page_views',
 'forum_page_views',
 'influence_page_views',
 'total_page_views',
 'fb_likes',
 'fb_talking_about',
 'twitter_followers',
 'twitter_following',
 'twitter_lists',
 'twitter_favourites',
 'twitter_statuses',
 'reddit_subscribers',
 'reddit_active_users',
 'reddit_posts_per_hour',
 'reddit_posts_per_day',
 'reddit_comments_per_hour',
 'reddit_comments_per_day',
 'code_repo_stars',
 'code_repo_forks',
 'code_repo_subscribers',
 'code_repo_open_pull_issues',
 'code_repo_closed_pu

### Frequencies

To get a list of available data (observations) frequencies

In [13]:
cc.frequencies

['min', 'h', 'd']

### Exchanges
To get a list of available exchanges

In [14]:
cc.exchanges

['Bitstamp',
 'Bittrex',
 'BTCE',
 'Okcoin',
 'BTER',
 'Poloniex',
 'Kraken',
 'Bitfinex',
 'Cex.io',
 'Yacuna',
 'Coinbase',
 'LocalBitcoins',
 'itBit',
 'HitBTC',
 'BTC38',
 'Coinfloor',
 'Huobi US',
 'LakeBTC',
 'Bit2C',
 'Coinsetter',
 'CCEX',
 'Coinse',
 'MonetaGo',
 'Gatecoin',
 'Yunbi',
 'Gemini',
 'CCEDK',
 'Cryptopia',
 'Exmo',
 'Yobit',
 'BitBay',
 'Korbit',
 'BTCMarkets',
 'Coincheck',
 'Bisq',
 'Vaultoro',
 'Mercado Bitcoin',
 'Unocoin',
 'Bitso',
 'BTCXIndia',
 'Paymium',
 'The Rock Trading',
 'bitFlyer',
 'Quoine',
 'Liqui',
 'Luno',
 'CryptoX',
 'EtherDelta',
 'TuxExchange',
 'CHBTC',
 'bitFlyerFX',
 'Livecoin',
 'Tidex',
 'CoinCorner',
 'BitMarket',
 'Coinone',
 'Bleutrade',
 'ETHEXIndia',
 'Surbitcoin',
 'Bithumb Korea',
 'FoxBit',
 'Zaif',
 'Jubi',
 'ViaBTC',
 'NovaExchange',
 'Waves.Exchange',
 'Binance',
 'Lykke',
 'Remitano',
 'Coinroom',
 'Abucoins',
 'BX.in.th',
 'Huobi Global',
 'OKX',
 'Gate.io',
 'TRUSTdex',
 'EXX',
 'Kucoin',
 'BitGrail',
 'Bit-Z',
 'BitFlip'

For more granular exchanges info

In [15]:
cc.get_exchanges_info()

,Id,Url,LogoUrl,ItemType,CentralizationType,InternalName,GradePoints,Grade,GradePointsSplit,AffiliateURL,...,FullAddress,Fees,DepositMethods,WithdrawalMethods,Sponsored,Recommended,Rating,SortOrder,TOTALVOLUME24H,DISPLAYTOTALVOLUME24H
Name,,,,,,,,,,,,,,,,,,,,,
Bitstamp,2431,/exchanges/bitstamp/overview,/media/37748052/bitstamp.png,"[Cryptocurrency, Fiat]",Centralized,Bitstamp,76.68,AA,"{'Legal': '12.9', 'KYCAndTransactionRisk': '14...",https://links.bitstamp.net/AoaR5N,...,Bitstamp Ltd\n5 New Street Square\nLondon EC4A...,"See Bitstamp's <a href=""https://www.bitstamp.n...",Cryptocurrency\nFiat\n\nSee Bitstamp's <a href...,Cryptocurrency\nFiat\n\nSee Bitstamp's <a href...,False,False,"{'One': 108, 'Two': 25, 'Three': 16, 'Four': 2...",3,{'BTC': 7297.28491469842},{'BTC': 'Ƀ 7.30 K'}
Bittrex,2434,/exchanges/bittrex/overview,/media/37746429/bittrex.png,"[Cryptocurrency, Stable Coins, Tokens, Fiat]",Centralized,BitTrex,65.34,BB,"{'Legal': '9.1', 'KYCAndTransactionRisk': '14'...",https://bittrex.com/account/register?referralC...,...,Bittrex LLC\nSeafirst Fifth Avenue Plaza\n800 ...,"See a <a href=""https://bittrexglobal.zendesk.c...","Cryptocurrency\nFiat\n\nSee a <a href=""https:/...","Cryptocurrency\nFiat\n\nSee a <a href=""https:/...",False,False,"{'One': 213, 'Two': 28, 'Three': 22, 'Four': 4...",5,{'BTC': 1408.9514294289},{'BTC': 'Ƀ 1.41 K'}
BTCE,2435,/exchanges/btce/overview,/media/19560/btce.png,[Cryptocurrency],Centralized,BTCE,0,-,"{'Legal': '0', 'KYCAndTransactionRisk': '0', '...",,...,Unknown,Trading Fee: 0.2%,BTC-e code:\t0%\nBTC: 0%\t\nLTC:\t0%\nNMC: 0%\...,BTC: 0.001 BTC\nLTC:\t0.001 LTC\nNMC: 0.1 NMC\...,False,False,"{'One': 6, 'Two': 0, 'Three': 2, 'Four': 2, 'F...",6,{'BTC': 0},{'BTC': 'Ƀ 0'}
Okcoin,2436,/exchanges/okcoin/overview,/media/39838353/okcoin.png,"[Cryptocurrency, Stable Coins, Fiat]",Centralized,OKCoin,72.22,A,"{'Legal': '14.1', 'KYCAndTransactionRisk': '9....",https://www.okcoin.com/join?channelId=600006168,...,"388 Market Street, \nSuite 1300, \nSan Francis...","See a <a href=""https://support.okcoin.com/hc/e...","Cryptocurrency\nFiat\n\nSee a <a href=""https:/...","Cryptocurrency\nFiat\n\nSee a <a href=""https:/...",False,False,"{'One': 8, 'Two': 5, 'Three': 4, 'Four': 4, 'F...",7,{'BTC': 2532.53435249073},{'BTC': 'Ƀ 2.53 K'}
BTER,2437,/exchanges/bter/overview,/media/19564/bter.png,[Cryptocurrency],Centralized,BTER,0,-,"{'Legal': '0', 'KYCAndTransactionRisk': '0', '...",https://bter.com,...,,Trading Fees: 0-0.2%,Cryptocurrency\nFiat\n\nWallet\nBank Transfer,Cryptocurrency\nFiat\n\nWallet: 0-1%\nBank Tra...,False,False,"{'One': 15, 'Two': 2, 'Three': 2, 'Four': 3, '...",8,{'BTC': 0},{'BTC': 'Ƀ 0'}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Uniswap (v3),943111,/exchanges/uniswap-v3/overview,/media/38553309/uniswapv3.png,"[Cryptocurrency, Stable Coins, Tokens]",Decentralized,uniswapv3,0,-,"{'Legal': '0', 'KYCAndTransactionRisk': '0', '...",https://app.uniswap.org/#/swap,...,Unknown,Uniswap v3 offers three different fee tiers: 0...,Cryptocurrency,Cryptocurrency,False,False,"{'One': 0, 'Two': 0, 'Three': 0, 'Four': 0, 'F...",322,{'BTC': 443.523402970925},{'BTC': 'Ƀ 443.52'}
PancakeSwap (v2),943129,/exchanges/pancakeswap-v2/overview,/media/38553321/pancakeswapv2.png,"[Cryptocurrency, Stable Coins, Tokens]",Decentralized,pancakeswap,0,-,"{'Legal': '0', 'KYCAndTransactionRisk': '0', '...",https://pancakeswap.finance/,...,Unknown,When users make a token swap (trade) on the ex...,Cryptocurrencies,Cryptocurrencies,False,False,"{'One': 0, 'Two': 0, 'Three': 0, 'Four': 0, 'F...",324,{'BTC': 0},{'BTC': 'Ƀ 0'}
Crypto.com,945093,/exchanges/cryptocom/overview,/media/39198190/cryptocom.png,"[Cryptocurrency, Derivatives, Stable Coins, To...",Centralized,cryptodotcom,64.37,BB,"{'Legal': '14.5', 'KYCAndTransactionRisk': '9....",https://crypto.com/exchange,...,"Level 7, Spinola Park, Triq Mikiel Ang Borg, S...","See the full list of <a href=""https://crypto.c...","See the full 

### Base URL
To get the API's base url

In [16]:
cc.base_url

'https://min-api.cryptocompare.com/data/'

### Maximum Observations per Call
To get the maximum observations (rows) per API call

In [17]:
cc.max_obs_per_call

2000

### Rate Limit
To get the number of API calls made and remaining

In [18]:
cc.rate_limit

,calls_made,calls_left
frequency,,
second,1,49
minute,7,2493
hour,7,24993
day,489,49511
month,12830,87170


## Data Request

Now that we know what data is available, we can send a data request to the data vendor's API. 

To do this, we must first import the DataRequest class and initialize a DataRequest Object containing all the relevant parameters for the data we want.

In [19]:
# import DataRequest
from cryptodatapy.data_requests.datarequest import DataRequest
# initialize DataRequest
data_req = DataRequest(tickers=['btc', 'eth'], fields=['close', 'active_addresses'])

We specify `btc` and `eth` tickers for `close` and `active_addresses` fields in our data request. Start and end dates, as well as data frequency, are set to defaults when not specified (i.e. earliest start date, latest end date and daily frequency).

For more details on data request parameters, see `datarequest` in our documentation API reference.

## Fetch Data

Once the parameters of our data request are set, `cryptodatapy` automatically converts those parameters to the CryptoCompare format saving you from spending time studying their API documentation.

To get the requested data, we can simply use the **fetch_data()** method with the data request parameters.

In [20]:
cc.fetch_data(data_req)

close  active_addresses
date       ticker                            
2009-01-03 BTC          NaN               1.0
2009-01-04 BTC          NaN               0.0
2009-01-05 BTC          NaN               0.0
2009-01-06 BTC          NaN               0.0
2009-01-07 BTC          NaN               0.0
...                     ...               ...
2022-06-26 ETH      1198.25          394830.0
2022-06-27 BTC     20718.16               NaN
           ETH      1190.73               NaN
2022-06-28 BTC     20561.37               NaN
           ETH      1170.51               NaN

[7443 rows x 2 columns]